In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from skorch import NeuralNetBinaryClassifier
import torch.nn as nn
import seaborn as sns
import pandas as pd
import numpy as np
import torch
import os
print(torch.__version__)

2.4.1+cu118


In [2]:
np.random.seed(123)  # Semente da aleatoriedade.
torch.manual_seed(123)  # Para os pesos da rede começar sempre iguais (BOA PRÁTICA).
torch.cuda.manual_seed(123)

baseFolder = os.path.join(os.path.dirname(os.getcwd()),"Bases")

previsoresData = os.path.join(baseFolder, "entradas_breast.csv")
classeData = os.path.join(baseFolder, "saidas_breast.csv")

previsores = pd.read_csv(previsoresData)  # Caracteristicas do tumor
classe = pd.read_csv(classeData)  # Maligno ou benigno

In [3]:
# CONVERTENDO PARA UM ARRAY NUMPY

previsores = np.array(previsores, dtype='float32')
classe = np.array(classe, dtype='float32').squeeze(1)

In [4]:
class classificador_torch(nn.Module):
    
    #DEFININDO REDE MAIS PERSONALIZADA !
    def __init__(self):
        # chama o construtor da classe nn.Module para garantir 
        # que a classe pai (superclasse) seja corretamente inicializada.
        super().__init__()

        # 30 -> 16 -> 16 -> 1
        
        self.dense0 = nn.Linear(30, 16)  # 1° camada.
        """
        Aqui, estamos inicializando os pesos da camada dense0 usando uma distribuição uniforme. Isso é feito manualmente.
        O PyTorch já inicializa os pesos automaticamente, mas nesse caso, estamos usando uma inicialização customizada.
        Para a utilização com o skortch isso é necessário.
        Estamos inicializando com pesos uniformes.
        """
        torch.nn.init.uniform(self.dense0.weight)

        self.activation0 = nn.ReLU() # Ativação.
        self.dense1 = nn.Linear(16, 16)
        torch.nn.init.uniform(self.dense1.weight)
        self.activation1 = nn.ReLU()
        self.dense2 = nn.Linear(16, 1)
        torch.nn.init.uniform(self.dense2.weight)
        self.output = nn.Sigmoid()
        
        
    def forward(self, x):
        # Como está em formato de classe, essa é a estrutura, precisamos fazer cada uma das camadas se comunicarem.
        x = self.dense0(x)
        x = self.activation0(x)
        x = self.dense1(x)
        x = self.activation1(x)
        x = self.dense2(x)
        x = self.output(x)
        
        return x
    

In [5]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  criterion=torch.nn.BCELoss, # Como o erro será calculado
                                                  optimizer=torch.optim.Adam,
                                                  lr=0.001,
                                                  optimizer__weight_decay=0.0001,
                                                  max_epochs=100,
                                                  batch_size=10,
                                                  train_split=False, # Não divide em dados de treino, pois vamos fazer na póoxima etapa usando validação cruzada.
                                                  )

In [ ]:
result = cross_val_score(classificador_sklearn, previsores, classe, cv=10, scoring='accuracy')

C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:18: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense0.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:22: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense1.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:25: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense2.weight)


  epoch    train_loss     dur
-------  ------------  ------
      1       37.1094  1.3299
      2       37.1094  0.2487
      3       37.1094  0.2635
      4       37.1094  0.3121
      5       37.1094  0.2207
      6       37.1094  0.1477
      7       37.1094  0.1308
      8       37.1094  0.1376
      9       37.1094  0.1306
     10       37.1094  0.1426
     11       37.1094  0.1218
     12       37.1094  0.1793
     13       37.1094  0.1933
     14       37.1094  0.2361
     15       37.1094  0.1785
     16       37.1094  0.1363
     17       37.1094  0.1142
     18       37.1094  0.1305
     19       37.1094  0.1446
     20       37.1094  0.1532
     21       37.1094  0.1653
     22       37.1094  0.1549
     23       37.1094  0.1216
     24       37.1094  0.1196
     25       13.5893  0.1712
     26        0.5466  0.1901
     27        0.5314  0.1612
     28        0.4981  0.2172
     29        0.4820  0.2258
     30        0.4769  0.2797
     31        0.4528  0.2108
     32   

C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:18: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense0.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:22: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense1.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:25: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense2.weight)


  epoch    train_loss     dur
-------  ------------  ------
      1       37.1094  0.2209
      2       37.1094  0.1166
      3       37.1094  0.1306
      4       37.1094  0.1611
      5       37.1094  0.1215
      6       37.1094  0.1145
      7       37.1094  0.1146
      8       37.1094  0.1255
      9       37.1094  0.1335
     10       37.1094  0.1858
     11       37.1094  0.2562
     12       37.1094  0.1886
     13       37.1094  0.2207
     14       37.1094  0.2001
     15       37.1094  0.1905
     16       37.1094  0.1746
     17       37.1094  0.1991
     18       37.1094  0.3059
     19       37.1094  0.1265
     20       37.1094  0.1356
     21       37.1094  0.1410
     22       37.1094  0.1405
     23       37.1094  0.1115
     24       37.1094  0.0835
     25       37.1094  0.1156
     26       14.5336  0.1156
     27        0.6825  0.1312
     28        0.6774  0.1445
     29        0.6740  0.1346
     30        0.6715  0.1971
     31        0.6696  0.1905
     32   

C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:18: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense0.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:22: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense1.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:25: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense2.weight)


...

      2       37.3047  0.1235
      3       37.3047  0.2459
      4       37.3047  0.1865
      5       37.3047  0.1446
      6       37.3047  0.1276
      7       37.3047  0.1205
      8       37.3047  0.1236
      9       37.3047  0.1402
     10       37.3047  0.1266
     11       37.3047  0.1352
     12       37.3047  0.1236
     13       37.3047  0.1366
     14       37.3047  0.0906
     15       37.3047  0.0996
     16       37.3047  0.1007
     17       37.3047  0.1076
     18       37.3047  0.1056
     19       37.3047  0.0996
     20       37.3047  0.1137
     21       37.3047  0.1035
     22       37.3047  0.1706
     23       35.1446  0.1713
     24        0.5579  0.1141
     25        0.5439  0.1103
     26        0.4934  0.1106
     27        0.4689  0.1331
     28        0.4497  0.1217
     29        0.4257  0.1176
     30        0.4166  0.1056
     31        0.3972  0.1442
     32        0.3884  0.1526
     33        0.3742  0.1432
     34        0.3626  0.1226
     35   

C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:18: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense0.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:22: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense1.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:25: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense2.weight)


      2       37.3047  0.1196
      3       37.3047  0.1066
      4       37.3047  0.1146
      5       37.3047  0.1226
      6       37.3047  0.1466
      7       37.3047  0.1358
      8       37.3047  0.1139
      9       37.3047  0.1225
     10       37.3047  0.1306
     11       37.3047  0.1641
     12       37.3047  0.1365
     13       37.3047  0.0893
     14       37.3047  0.1166
     15       37.3047  0.1153
     16       37.3047  0.1086
     17       37.3047  0.1315
     18       37.3047  0.1301
     19       37.3047  0.1216
     20       37.3047  0.1406
     21       37.3047  0.1166
     22       37.3047  0.1056
     23       37.3047  0.1421
     24       37.3047  0.1376
     25       16.2481  0.1743
     26        0.5808  0.1607
     27        0.5693  0.1332
     28        0.5604  0.1196
     29        0.5554  0.1344
     30        0.5529  0.1156
     31        0.5490  0.1219
     32        0.5447  0.1631
     33        0.4927  0.1197
     34        0.4586  0.1596
     35   

C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:18: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense0.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:22: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense1.weight)
C:\Users\wiwil\AppData\Local\Temp\ipykernel_19396\83687862.py:25: FutureWarning: `nn.init.uniform` is now deprecated in favor of `nn.init.uniform_`.
  torch.nn.init.uniform(self.dense2.weight)


      2       37.3047  0.1672
      3       37.3047  0.1559
      4       37.3047  0.1677
      5       37.3047  0.1446
      6       37.3047  0.2447
      7       37.3047  0.1386
      8       37.3047  0.1413
      9       37.3047  0.1717
     10       37.3047  0.1221
     11       37.3047  0.1126
     12       37.3047  0.1276
     13       37.3047  0.1332
     14       37.3047  0.1331
     15       37.3047  0.1496
     16       37.3047  0.1613
     17       37.3047  0.1345
     18       37.3047  0.2112
     19       37.3047  0.2644
     20       37.3047  0.1496
     21       37.3047  0.1497
     22       37.3047  0.2012
     23       12.7908  0.2389
     24        0.5656  0.2048
     25        0.5195  0.1605
     26        0.4931  0.1931
     27        0.4548  0.1611
     28        0.4263  0.1732
     29        0.4036  0.1556
     30        0.3794  0.2102
     31        0.3631  0.2968
     32        0.3460  0.2007
     33        0.3366  0.1463
     34        0.3270  0.1526
     35   

In [7]:
media = result.mean()
print("media",media)
print("desvio", result.std())

media 0.806704260651629
desvio 0.12251488513561204
